In [ ]:
# Ячейка 1: Установка базовых компонентов
!apt-get update -qq
!apt-get install -y -qq openssh-client wget git

# Установка dotnet для SwarmUI
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0


In [ ]:
# Ячейка 2: Клонирование SwarmUI
!git clone https://github.com/mcmonkeyprojects/SwarmUI.git
%cd SwarmUI

# Установка SwarmUI
!chmod +x ./launch-linux.sh


In [ ]:
# Ячейка 3: Создание директорий для моделей
!mkdir -p /kaggle/temp/models/diffusion_models
!mkdir -p /kaggle/temp/models/vae
!mkdir -p /kaggle/temp/models/clip

# Загрузка FLUX Kontext Dev модели (выберите один вариант)
# Вариант 1: Полная модель (требует больше VRAM)
!wget -O /kaggle/temp/models/diffusion_models/flux1-kontext-dev.safetensors \
    "https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev/resolve/main/flux1-kontext-dev.safetensors"

# Вариант 2: FP8 версия (рекомендуется для Kaggle)
!wget -O /kaggle/temp/models/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors \
    "https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors"

# Загрузка необходимых компонентов
!wget -O /kaggle/temp/models/vae/ae.safetensors \
    "https://huggingface.co/black-forest-labs/FLUX.1-dev/resolve/main/ae.safetensors"

!wget -O /kaggle/temp/models/clip/clip_l.safetensors \
    "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors"

!wget -O /kaggle/temp/models/clip/t5xxl_fp16.safetensors \
    "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors"


In [ ]:
# Ячейка 4: Настройка Pinggy (запускайте в отдельной ячейке)
import subprocess
import time
from threading import Thread

def start_pinggy():
    """Запуск Pinggy туннеля"""
    cmd = ["ssh", "-p", "443", "-o", "StrictHostKeyChecking=no", 
           "-R0:localhost:7801", "a.pinggy.io"]
    
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, 
                              stderr=subprocess.STDOUT, text=True)
    
    for line in process.stdout:
        print(line.strip())
        if "https://" in line and "pinggy" in line:
            print(f"\n🌐 ВНЕШНЯЯ ССЫЛКА: {line.strip()}")
        if "tcp://" in line:
            print(f"🔗 TCP доступ: {line.strip()}")

# Запуск туннеля в фоновом режиме
pinggy_thread = Thread(target=start_pinggy)
pinggy_thread.daemon = True
pinggy_thread.start()


In [ ]:
# Ячейка 5: Запуск SwarmUI сервера
import subprocess
import time

def start_swarmui():
    """Запуск SwarmUI сервера"""
    cmd = ["./launch-linux.sh", "--launch_mode", "none", "--host", "0.0.0.0"]
    
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, 
                              stderr=subprocess.STDOUT, text=True, cwd="/kaggle/working/SwarmUI")
    
    for line in process.stdout:
        print(line.strip())
        if "starting web server" in line.lower():
            print("\n✅ SwarmUI запущен! Переходите по ссылке Pinggy")
            break

start_swarmui()
